In [1]:
from farm.utils import initialize_device_settings
from farm.modeling.tokenization import Tokenizer
from farm.data_handler.processor import TextClassificationProcessor, SquadProcessor
from farm.data_handler.data_silo import DataSilo
from farm.eval import Evaluator
from farm.modeling.adaptive_model import AdaptiveModel
from pathlib import Path
from farm.infer import Inferencer



In [2]:
import pandas as pd
df = pd.read_csv("/home/graphn/repositories/you_conscious/dl_xp/data/category/train_category.tsv",sep="\t")
label_list_1 = df["label"].tolist()
label_list_1 = list(set(label_list_1))
df = pd.read_csv("/home/graphn/repositories/you_conscious/dl_xp/data/category/test_category.tsv",sep="\t")
label_list_2 = df["label"].tolist()
label_list_2 = list(set(label_list_2))
label_list = label_list_1 + label_list_2 + [""]
label_list = list(set(label_list_1+label_list_2)) 
texts = df["text"].tolist()
y_true  = df["label"].tolist()

In [3]:
##########################
########## Settings
##########################
device, n_gpu = initialize_device_settings(use_cuda=False)
lang_model = "/home/graphn/repositories/you_conscious/dl_xp/trained_models/category_v2"
do_lower_case = False
batch_size = 20

data_dir = Path("../data")
evaluation_filename = "test_all_countries.tsv"
label_list = label_list
metric = "f1_macro"


04/13/2021 07:04:32 - INFO - farm.utils -   Using device: CPU 
04/13/2021 07:04:32 - INFO - farm.utils -   Number of GPUs: 0
04/13/2021 07:04:32 - INFO - farm.utils -   Distributed Training: False
04/13/2021 07:04:32 - INFO - farm.utils -   Automatic Mixed Precision: None


In [4]:
# 9. Load it & harvest your fruits (Inference)
save_dir =  "/home/graphn/repositories/you_conscious/dl_xp/trained_models/category_v2"
basic_texts = []
for t in texts:
    basic_texts.append({"text":t})
print(len(basic_texts))
model = Inferencer.load(save_dir, gpu=True, return_class_probs=False,task_type="text_classification")
result = model.inference_from_dicts(dicts=basic_texts)

04/13/2021 07:04:32 - INFO - farm.utils -   Using device: CUDA 
04/13/2021 07:04:32 - INFO - farm.utils -   Number of GPUs: 1
04/13/2021 07:04:32 - INFO - farm.utils -   Distributed Training: False
04/13/2021 07:04:32 - INFO - farm.utils -   Automatic Mixed Precision: None
04/13/2021 07:04:32 - INFO - farm.modeling.language_model -   
04/13/2021 07:04:32 - INFO - farm.modeling.language_model -   LOADING MODEL
04/13/2021 07:04:32 - INFO - farm.modeling.language_model -   =============
04/13/2021 07:04:32 - INFO - farm.modeling.language_model -   Model found locally at /home/graphn/repositories/you_conscious/dl_xp/trained_models/category_v2


11866


04/13/2021 07:04:33 - INFO - farm.modeling.language_model -   Loaded /home/graphn/repositories/you_conscious/dl_xp/trained_models/category_v2
04/13/2021 07:04:33 - INFO - farm.modeling.adaptive_model -   Found files for loading 1 prediction heads
04/13/2021 07:04:33 - WARNING - farm.modeling.prediction_head -   `layer_dims` will be deprecated in future releases
04/13/2021 07:04:33 - INFO - farm.modeling.prediction_head -   Prediction head initialized with size [768, 134]
04/13/2021 07:04:33 - INFO - farm.modeling.prediction_head -   Loading prediction head from /home/graphn/repositories/you_conscious/dl_xp/trained_models/category_v2/prediction_head_0.bin
04/13/2021 07:04:35 - INFO - farm.modeling.tokenization -   Loading tokenizer of type 'BertTokenizerFast'
04/13/2021 07:04:35 - INFO - farm.data_handler.processor -   Initialized processor without tasks. Supply `metric` and `label_list` to the constructor for using the default task or add a custom task later via processor.add_task()
04

04/13/2021 07:04:36 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
04/13/2021 07:04:36 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
04/13/2021 07:04:36 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
04/13/2021 07:04:36 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
Inferencing Samples:  12%|█▎        | 5/40 [00:00<00:02, 15.89 Batches/s]04/13/2021 07:04:36 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
04/13/2021 07:04:36 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
04/13/2021 07:04:36 - WARNING - farm.data_handler.processor -   Currently no support in Processor for returning problematic ids
Inferencing Samples:  20%|██   

Inferencing Samples: 100%|██████████| 25/25 [00:01<00:00, 24.94 Batches/s]


In [5]:
import numpy as np
predictions=[]
i = 0
for batch in result:
    for x in batch["predictions"]:
        if True:
            prediction = x["label"]
            predictions.append(prediction)
        #probabilities = x["probability"]
        #position = np.argmax(probabilities)
        #print(id_2_labels[position],y_true[i])
        #labels = list(id_2_labels.values())
        #predictions.append({label:prob for label,prob in zip(probabilities,labels)})
        i+=1

In [10]:
predictions[0:2]

['Accessoires > Damen > Taschen > Rucksäcke',
 'Bekleidung > Damen > Jacken & Mäntel > Jacken']

In [11]:
from sklearn.metrics import classification_report
def evaluate(predictions,Y_test):
    return classification_report(predictions, Y_test,output_dict=True)#, target_names=target_names

In [12]:
evaluation=evaluate(predictions,y_true)

/home/graphn/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/graphn/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/graphn/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/graphn/.local/lib/python3.8/site-packages/

In [13]:
precisions = []
recalls = []
f1s = []
supports = []
labels = []
for k,v in evaluation.items():
    if type(v) != float: #!="accuracy":
        labels.append(k)
        precisions.append(v["precision"])
        recalls.append(v["recall"])
        f1s.append(v["f1-score"])
        supports.append(v["support"])
x = pd.DataFrame({"labels":labels,"precision":precisions,"recall":recalls,"f1_score":f1s,"support":supports})
x.to_csv("category_v2.tsv",sep="\t")